In [1]:
import transformers
from transformers import AutoTokenizer
import datasets
import tqdm 
for_ipynb_run = "/gpfsdswork/projects/rech/btm/uei84ht/"
import pandas as pd
from nltk.tokenize import word_tokenize, RegexpTokenizer

In [21]:
hggfc_model_name = 'bert-base-uncased'#'EleutherAI/gpt-j-6B'
dataset_subset = 'scotus'#'ildc'#'ecthr_a'

In [3]:
model_path = "/gpfsdswork/dataset/HuggingFace_Models/"+hggfc_model_name
tokenizer = AutoTokenizer.from_pretrained(model_path)
nltk_word_tokenizer = RegexpTokenizer(r'\w+')

In [22]:
dataset = datasets.load_from_disk(f"/gpfsdswork/dataset/HuggingFace/lex_glue/{dataset_subset}")
train_data = dataset['train'].to_pandas()
validation_data = dataset['validation'].to_pandas()
test_data = dataset['test'].to_pandas()
text_column = train_data.columns[0]
label_column = train_data.columns[1]


In [13]:
# only for ildc
for_ipynb_run = "/gpfsdswork/projects/rech/btm/uei84ht/"
data = pd.read_csv(for_ipynb_run+"LEGAL-PE/SIGIR_experiments/datasets/ILDC/ILDC_multi.csv")
data_s = pd.read_csv(for_ipynb_run+"LEGAL-PE/SIGIR_experiments/datasets/ILDC/ILDC_single.csv")
test_data_exp = pd.read_csv(for_ipynb_run+"LEGAL-PE/SIGIR_experiments/datasets/ILDC/anno_dataset.csv")
train_data = data.query("split=='train'")
train_data_s = data_s.query("split=='train'")
validation_data = data.query("split=='dev'")
test_data = data.query("split=='test'")
#exp_data = pd.read_csv(r"LEGAL-PE/SIGIR_experiments/datasets/ILDC/anno_dataset.csv")
#exp_data.rename(columns={'text_y':'text'}, inplace=True)

train_data = train_data.append(train_data_s, ignore_index=True)
del train_data_s, data_s, data
text_column = 'text'

/tmp/ipykernel_4059835/2807804493.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data = train_data.append(train_data_s, ignore_index=True)


## Get average token counts

In [7]:
from tqdm import tqdm
def get_average_token_count(df, column_name, tokenizer, nltk_word_tokenizer = None):
    list_tokens = []
    total_len = 0
    for i in tqdm(range(len(df[column_name]))):
        texts = df[text_column].iloc[i]
        if nltk_word_tokenizer == None:
            tokens = tokenizer.tokenize(texts)
        else:
            tokens = nltk_word_tokenizer.tokenize(texts)
        total_len = total_len+len(tokens)
        list_tokens.append(len(tokens))
    return total_len/len(df[column_name]), list_tokens
def concatenate_list_of_texts(data, column: str):
    for i in range(len(data)):
        conc_text = ""
        for j in range(len(data[column][i])):
            conc_text+=data[column][i][j][4:]
        data[column][i] = conc_text
    return data

In [8]:
train_data = concatenate_list_of_texts(train_data, column = "text")
validation_data = concatenate_list_of_texts(validation_data, column = "text")
test_data = concatenate_list_of_texts(test_data, column = "text")
#test_data_exp = concatenate_list_of_texts(test_data_exp, column = "text_y")

/tmp/ipykernel_1597883/2285960699.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column][i] = conc_text


In [14]:

train_c,max_tr = get_average_token_count(train_data, text_column, tokenizer, nltk_word_tokenizer=nltk_word_tokenizer)
validate_c,max_va = get_average_token_count(validation_data, text_column, tokenizer, nltk_word_tokenizer=nltk_word_tokenizer)
test_c,max_te = get_average_token_count(test_data, text_column, tokenizer, nltk_word_tokenizer=nltk_word_tokenizer)

if dataset_subset=='ildc':
    test_exp, max_exp = get_average_token_count(test_data_exp, column_name='text_y', tokenizer=tokenizer, nltk_word_tokenizer = nltk_word_tokenizer)
    

100%|██████████| 56/56 [00:00<00:00, 1037.58it/s]


In [15]:
print(f"avg: {train_c}, max: {max(max_tr)}")
print(f"avg: {validate_c}, max: {max(max_va)}")
print(f"avg: {test_c}, max: {max(max_te)}")
if dataset_subset=='ildc':
    print(f"avg: {test_exp}, max: {max(max_exp)}")

avg: 3326.086848369754, max: 415160
avg: 3792.339034205231, max: 42447
avg: 3893.00593276203, max: 40722
avg: 3091.964285714286, max: 20569


Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x14e2941e9d50>>
Traceback (most recent call last):
  File "/gpfslocalsup/pub/anaconda-py3/2023.03/envs/pytorch-gpu-2.0.0+py3.10.9/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3872, in atexit_operations
    self._atexit_once()
  File "/gpfslocalsup/pub/anaconda-py3/2023.03/envs/pytorch-gpu-2.0.0+py3.10.9/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3855, in _atexit_once
    self.history_manager.end_session()
  File "/gpfslocalsup/pub/anaconda-py3/2023.03/envs/pytorch-gpu-2.0.0+py3.10.9/lib/python3.10/site-packages/IPython/core/history.py", line 579, in end_session
    self.writeout_cache()
  File "<decorator-gen-17>", line 2, in writeout_cache
  File "/gpfslocalsup/pub/anaconda-py3/2023.03/envs/pytorch-gpu-2.0.0+py3.10.9/lib/python3.10/site-packages/IPython/core/history.py", line 60, in only_when_enab

In [72]:
from collections import Counter
#Counter(max_tr)

In [71]:
max_tr.index(501275)

19232

In [74]:
train_data['text'][19232]

' P. C  135 OF 1970 Appellants His Holiness Kesavananda Bharati Sripadagalvaru and Ors. Vs. Respondent State of Kerala and Anr. Decided On 24.04.1973 Honble Judges M. Sikri, C.J., A.N. Grover, A.N. Ray, D.G. Palekar, H.R. Khanna, J.M. Shelat, K.K. Mathew, K.S. Hegde M.H. Beg, P. Jaganmohan Reddy, S.N. Dwivedi and Y.V. Chandrachud, JJ. JUDGMENT M. Sikri, C.J. I propose to divide my judgment into eight parts. Part I will deal with Introduction Part II with interpretation of Golakhnath case Part III with the interpretation of the original Article 368, as it existed prior to its amendment Part IV with the validity of the Constitution  Twenty-fourth Amendment  Act Part V with the validity of Section 2 of the Constitution  Twenty-fifth Amendment  Act Part VI with the validity of Section 3 of the Constitution  Twenty-fifth Amendment  Act Part VII with Constitution  Twenty- ninth Amendment  Act and Part VIII with companyclusions. PART I-Introduction All the six writ petitions involve companymo

## Get class distributions

In [6]:
def get_vocabulary(
    df,
    column,
    problem_type: str = 'multi_label'
    ):
    vocabulary = []
    if problem_type == 'multi_label':
        for doc in range(len(df)):
            labels = df[column][doc]
            for label in labels:
                if label not in vocabulary:
                    vocabulary.append(label)
    elif problem_type == 'multi_class':
        for doc in range(len(df)):
            label = df[column][doc]
            if label not in vocabulary:
                vocabulary.append(label)
    vocabulary.sort()
    return vocabulary

In [27]:
def get_class_dist(
    df,
    column,
    problem_type: str = 'multi_label'
    ):
    vocab = get_vocabulary(df, column, problem_type)
    class_dict = {}
    for i in vocab:
        class_dict[i] = 0
    if problem_type == 'multi_label':
        for doc in range(len(df)):
            labels = df[column][doc]
            for label in labels:
                class_dict[label]+=1
    elif problem_type == 'multi_class':
        for doc in range(len(df)):
            label = df[column][doc]
            class_dict[label]+=1
    return class_dict
    

In [30]:
data = test_data
problem_type = 'multi_class'
column = 'label'
print(get_class_dist(data, column=column, problem_type=problem_type))

{0: 372, 1: 222, 2: 88, 3: 51, 4: 28, 5: 17, 6: 24, 7: 260, 8: 200, 9: 83, 10: 15, 11: 37, 12: 3}
